In [1]:
from os import listdir
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
import numpy as np
from tqdm import tqdm
import wikipedia
from nltk.metrics.distance import edit_distance

In [2]:
MISSING = ""

# Loading local data

In [3]:
#Récupére les données sur le PC pour éviter de scrapper à chaque fois
files = listdir("../chart")

dfs_chart = {}
for f in files:
    year = f.split("_")[0]
    month = f.split("_")[1].split(".")[0]

    if dfs_chart.get(year) is None:
        dfs_chart[year] = {}
        
    dfs_chart[year][month] = pd.read_csv("../chart/" + f)

In [4]:
dfs_chart["2006"]["Janvier"].head(3)

,Rank,Artist,Music
0,1,juanes,la camisa negra
1,2,madonna,hung up
2,3,johnny hallyday,mon plus beau noël


In [52]:
artist = pd.read_csv("../artist.csv")

In [53]:
artist

,Artist,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays,Summary
0,ana,Ana,NaN,NaN,NaN,NaN,NaN,NaN
1,le manège enchanté,Le Manège enchanté,NaN,france,NaN,française,france,le manège enchanté est une série télévisée d'a...
2,raphael,Raphael (chanteur français),"7 novembre 1975 (47 ans)paris 8e, france",NaN,NaN,française,NaN,"raphaël haroche, dit raphael,,,, est un auteur..."
3,pete yorn,Pete Yorn,27 juillet 1974 (48 ans)montville,NaN,NaN,américaine,NaN,peter yorn dit pete yorn est un musicien améri...
4,pzk,PZK,NaN,france,NaN,française,france,pzk est un groupe de pop et musique électroniq...
...,...,...,...,...,...,...,...,...
2241,nassi,Nassi,NaN,NaN,NaN,NaN,NaN,nassi (hébreu : נָשִׂיא) est un terme hébreu s...
2242,niall horan,Niall Horan,"13 septembre 1993 (29 ans)mullingar, irlande",NaN,NaN,NaN,NaN,"niall james horan, né le 13 septembre 1993 à m..."
2243,h.e.r.,H.E.R.,"27 juin 1997 (25 ans)vallejo, états-unis",NaN,NaN,NaN,NaN,"gabriella wilson, dite h.e.r., née le 27 juin ..."
2244,erikel,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Supression des dernières columns

In [56]:
artist.drop(columns=["Naissance", "Pays d'origine", "Origine", "Pays", "Nationalité"], inplace=True)
artist.head(3)

,Artist,Artist_wiki,Summary
0,ana,Ana,NaN
1,le manège enchanté,Le Manège enchanté,le manège enchanté est une série télévisée d'a...
2,raphael,Raphael (chanteur français),"raphaël haroche, dit raphael,,,, est un auteur..."


In [57]:
artist.drop(columns=["Artist_wiki"], inplace=True)

# Scrapping chart

In [23]:
years = [str(y) for y in np.arange(2004, 2023, 1)]
weeks = [int(v) for v in np.linspace(1, 52, 12)]
mois = ["Janvier", "Fevrier", "Mars", "Avril", "Mai", "Juin", "Juillet", "Aout", "Septembre", "Octobre", "Novembre", "Decembre"]

In [24]:
dfs_chart = {y : {} for y in years}
for y in tqdm(years):
    for k, w in enumerate(weeks):
        url = f"https://acharts.co/france_singles_top_100/{y}/{w}"
        rq = get(url)
        m = mois[k]
        if rq.ok:
            soup = BeautifulSoup(rq.text)

            chart = soup.find("table", {"id" : "ChartTable"})
            trs = chart.findAll("tr")[1:]

            data = []
            for tr in trs:
                music_name = tr.find("span", {"itemprop" : "name"}).text.lower()
                rank = tr.find("span", {"itemprop" : "position"}).text.lower()
                artist_name = tr.find("span", {"itemprop" : "byArtist"}).text[2:-1].lower()
                data.append([str(rank), str(artist_name), str(music_name)])

            df = pd.DataFrame(data, columns=["Rank", "Artist", "Music"])
            dfs_chart[y][m] = df

100%|██████████| 19/19 [02:36<00:00,  8.26s/it]


# Cleanning chart

In [5]:
names = [
    ("Djadja", "Djadja et Dinaz"),
    ("Lorie", "Lorie Pester"),
    ("-m-", "Matthieu Chedid"),
    ("Priscilla", "Priscilla Betti"),
    ("I Am", "IAM"),
    ("Sofiane", "Sofiane Zermani"),
    ("Justice", "Justice (groupe)"),
    ("Soma Riba", "Collectif Métissé"),
    ("Fresh", "Fresh la Peufra"),
    ("Rosalia", "Rosalía"),
    ("Italo Brothers", "ItaloBrothers"),
    ("Far\*east Movement", "Far East Movement"),
    ("Odyssey", "Odyssey (groupe)"),
    ("1789", "1789 : Les Amants de la Bastille"),
    ("Clemence", "Clémence Saint-Preux"),
    ("Rose", "Rose (chanteuse)"),
    ("Laeti", "Laetitia Kerfa"),
    ("La Troupe", "Mozart, l'opéra rock"),
    ("Victoria", "Victoria Sio"),
    ("Christine And The Queens", "Redcar (artiste)"),
    ("Earth and Wind And Fire", "Earth, Wind And Fire"),
    ("Jean Roch", "Jean-Roch"),
    ("Keen V", "Keen'V"),
    ("Dinor", "Dinor RDT"),
    ("Koba La D", "Koba LaD"),
    ("scotts", "Travis Scott"),
    ("louane emera", "louane"),
    ("maitre gims", "gims"),
    ("p!nk", "pink"),
    ("Eva", "Eva Queen"),
    ("dj tiësto", "tiesto"),
    ("Karol", "Karol G"),
    ("Khaled", "DJ Khaled"),
    ("Black Eyed Peas", "The Black Eyed Peas"),
    ("L.E.J", "LEJ"),
    ("Disiz la peste", "Disiz"),
    ("k'maro", "k. maro"),
    ("shin sekaï", "The Shin Sekaï"),
    ("the niro", "niro"),
    ("r.i.o.", "rio"),
    ("zayn", "zayn malik"),
    ("dimitri vegas", "dimitri vegas & like mike"),
    ("cauet", "Sébastien Cauet"),
    ("do", "the do"),
    ("les filles", "Aurélie Konaté")
]

replace_words = [
    ("\x9c", "oe"),
    ("œ", "oe"),
    ("\$", "S"),
]

for k1 in dfs_chart.keys():
    for k2 in dfs_chart[k1].keys():
        dfs_chart[k1][k2]["Artist"] = dfs_chart[k1][k2]["Artist"].str.split(" x ").str[0]
        dfs_chart[k1][k2]["Artist"] = dfs_chart[k1][k2]["Artist"].str.split(" - ").str[-1]
        dfs_chart[k1][k2]["Artist"] = dfs_chart[k1][k2]["Artist"].str.split(" \+ ").str[0]

        dfs_chart[k1][k2]["Artist"] = dfs_chart[k1][k2]["Artist"].apply(lambda x: "star academy" if "star academy" in x else x)
        
        for n1, n2 in names:
            dfs_chart[k1][k2]["Artist"] = dfs_chart[k1][k2]["Artist"].apply(lambda x: n2 if x == n1 else x)

        for n1, n2 in replace_words:
            dfs_chart[k1][k2]["Artist"] = dfs_chart[k1][k2]["Artist"].str.replace(n1.lower(), n2.lower(), regex=False)
        

# Scrapping birth

In [ ]:
def find_title_in_wikipedia(title, pourcentage=0.3):
    words = ["(chanteur)", "(chanteuse)", "(groupe)", "(rappeur)", "(rappeuse)", "(musicien)", "(chanteur français)", "(france)", "(producteur)", "(artiste)", "(groupe de musique)"]

    wikipedia.set_lang("fr")
    results = wikipedia.search(title, results=10)
    distance = []
    if len(results) > 0:
        for element in results:
            if any((w in element.lower()) and (edit_distance(element.lower().split(" (")[0].strip(), title.lower().strip())/len(title) < pourcentage) for w in words):
                return element

            distance.append(edit_distance(title.lower().strip(), element.lower().strip()))

        return results[np.argmin(distance)] if min(distance)/len(title) < pourcentage else MISSING

In [ ]:
def wiki_birth(title):
    cols = ["Naissance", "Pays d'origine", "Origine", "Nationalité", "Pays", "Summary"]
    nats = ["franco", "français", "belge", "canadien", "libanais", "réunionnais"]
    dic = {w : MISSING for w in cols}

    if title == MISSING:
        return dic

    url = f"https://fr.wikipedia.org/wiki/{title}"
    rq = get(url)

    if not rq.ok:
        return dic
    
    soup = BeautifulSoup(rq.text)
    tables = soup.findAll("table")

    for table in tables:
        trs = table.findAll("tr")

        for tr in trs:
            th = tr.find("th")

            if th is not None:
                for w in cols:
                    if w in th.text:
                        td = tr.find("td")
                        if td is not None:
                            dic[w] = td.text.strip().lower()

    wikipedia.set_lang("fr")
    try:
        summary = wikipedia.summary(title, sentences=1)
        dic["Summary"] = summary.lower().strip()

        if dic["Nationalité"] == MISSING:
            for w in nats:
                if w in summary:
                    dic["Nationalité"] = w.lower().strip()
                    return dic
    except:
        pass

    return dic

In [ ]:
#Récupére tous les artistes uniques
artist = []
for k1 in dfs_chart.keys():
    for k2 in dfs_chart[k1].keys():
        artist += dfs_chart[k1][k2]["Artist"].tolist()
    
artist = pd.DataFrame(list(set(artist)), columns=["Artist"])

In [ ]:
artist["Artist_wiki"] = artist["Artist"].apply(lambda x : find_title_in_wikipedia(x, 0.2)) #Trouve les pages wikipedia de chaque artistes

In [ ]:
birth_dic = artist["Artist_wiki"].apply(wiki_birth) #Cherche les infos de naissance sur les pages wikipedia

c:\Users\quent\AppData\Local\Programs\Python\Python310\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\quent\AppData\Local\Programs\Python\Python310\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [ ]:
#Transforme les infos trouver sur wikipédia en dataframe
dfs_birth = []
for dic in birth_dic:
    dfs_birth.append(pd.DataFrame(dic, index=[0]))
birth = pd.concat(dfs_birth, ignore_index=True)

In [ ]:
#Merge les infos de naissance avec les infos des artistes
artist = artist.merge(birth, left_index=True, right_index=True)
artist = artist.set_index("Artist")

In [ ]:
#On ajoute manuellement les informations pour les artistes importants qui n'ont pas été trouvé sur wikipedia
manuel_names = [
    ("sound of legend", "MANUEL", MISSING, MISSING, MISSING, "française", MISSING, MISSING),
    ("al. hy", "MANUEL", "15 novembre 1993", MISSING, MISSING, "française", MISSING, "15 novembre 1993 (Âge: 29 ans), Saint-Amand-les-Eaux"),
    ("glk", "MANUEL", MISSING, MISSING, MISSING, "française", MISSING, "Originaire de Bobigny, en Seine-Saint-Denis"),
    ("paul glaeser", "MANUEL", "1963", MISSING, MISSING, "française", MISSING, MISSING),
    ("bolémvn", "MANUEL", "29 novembre 1996", MISSING, MISSING, "française", MISSING, "Bryan Mounkala (né le 29 novembre 1996) mieux connu sous le nom de Bolémvn est un rappeur français d'Évry, Essonne, Île-de-France, France."),
    ("paul glaeser", "MANUEL", "1963", MISSING, MISSING, "française", MISSING, MISSING),
    ("1pliké140", "MANUEL", "1963", MISSING, MISSING, "française", MISSING, "1PLIKÉ140 est un jeune rappeur français originaire de Clamart (92) en banlieue parisienne."),
    ("jérôme collet", "MANUEL", MISSING, MISSING, MISSING, "française", MISSING, MISSING),
    ("funnybear", "MANUEL", MISSING, MISSING, MISSING, "française", MISSING, MISSING),
    ("tom snare", "MANUEL", MISSING, MISSING, MISSING, "française", MISSING, "Xavier Decanter, mieux connu sous son nom de scène Tom Snare, est un DJ et producteur de disques français originaire de Dunkerque."),
    ("funnybear", "MANUEL", "22 janvier 1991", MISSING, MISSING, "française", MISSING, "22 janvier 1991 (Âge: 31 ans), Hyères"),
    ("landy", "MANUEL", "2000", MISSING, MISSING, "française", MISSING, "Dylan Sylla Gahoussou, « Landy » de son nom de scène, est né dans le XIXe arrondissement de Paris"),
    ("dry", "MANUEL", "19 novembre 1977", MISSING, MISSING, "française", MISSING, "Landry Delica a grandi à Sevran en Seine-Saint-Denis"),
    ("sasso", "MANUEL", MISSING, MISSING, MISSING, "française", MISSING, "Né à Vénissieux d'un père togolais et d'une mère marocaine"),
    ("neïman", "MANUEL", MISSING, MISSING, MISSING, "guyanaise", MISSING, "NEÏMAN est un chanteur et toaster français de dancehall et de reggae-soul né RAY NEÏMAN en Guyane."),
    ("isk", "MANUEL", "20 mars 2003", MISSING, MISSING, "française", MISSING, "ISK, de son vrain nom Kais Ben Baccar, est un rappeur français d'origine tunisienne, né au Canada. Habitant La Ferté-sous-Jouarre (77), il est membre du label Bendo 11 Records sous la division rap GrandLine."),
    ("yaro", "MANUEL", "1996", MISSING, MISSING, "française", MISSING, "Yaro, anciennement Sirsy, est un rappeur français originaire de la ville de Yerres dans l'Essonne.")
]

manuel_artist = pd.DataFrame(manuel_names, columns=["Artist", "Artist_wiki", "Naissance", "Pays d'origine", "Origine", "Nationalité", "Pays", "Summary"]).set_index("Artist")
artist.drop(index=manuel_artist.index, inplace=True)
artist = pd.concat([artist, manuel_artist])

In [ ]:
artist

,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays,Summary
Artist,,,,,,,
kareen antonn,Kareen Antonn,"24 mai 1980 (42 ans)nogent-sur-marne, (val-de-...",,,français,,"kareen antonn, de son vrai nom kareen antonopo..."
j five,J-Five,"1982los angeles, californie, états-unis",,,,,"j-five, de son vrai nom jonathan kovacs, né le..."
claudio capéo,Claudio Capéo,10 janvier 1985 (37 ans)mulhouse (haut-rhin),,,france,,"claudio capéo, de son vrai nom claudio ruccolo..."
loren allred,Loren Allred,"7 septembre 1989 (33 ans)pittsburgh, pennsylvanie",,,,,
patxi,,,,,,,
...,...,...,...,...,...,...,...
dry,MANUEL,19 novembre 1977,,française,,,Landry Delica a grandi à Sevran en Seine-Saint...
sasso,MANUEL,,,française,,,Né à Vénissieux d'un père togolais et d'une mè...
neïman,MANUEL,,,guyanaise,,,NEÏMAN est un chanteur et toaster français de ...


# Cleanning Wikipédia results

In [ ]:
def cleanning(data, replace_words):
    new_df = data.copy()
    new_df = new_df.str.lower()
    for w1, w2 in replace_words:
        new_df[new_df.str.contains(w1)] = w2

    return new_df

In [ ]:
replace_words = [
    ("français", "française"),
    ("franco", "française"),
    ("canadien", "canadienne"),
    ("américain", "américaine"),
    ("états-unis", "américaine"),
    ("algérien", "algérienne"),
    ("brésil", "brésilienne"),
    ("france", "française"),
    ("marocain", "marocaine"),
    ("anglais", "britannique"),
    ("royaume-unis", "britannique"),
    ("britannique", "britannique"),
    ("espagn", "espagnole"),
    ("italie", "italienne"),
    ("japon", "japonaise"),
    ("corée", "coréenne"),
    ("israél", "israélienne"),
    ("royaume-uni", "britannique"),
    ("suède", "suédoise"),
    ("allemand", "allemande"),
    ("dominicain", "dominicaine"),
    ("island", "islandaise"),
]

artist["Nationalité"] = cleanning(artist["Nationalité"], replace_words)
artist["Origine"] = cleanning(artist["Origine"], replace_words)

In [ ]:
replace_words = [
    ("france", "france"),
    ("états-unis", "états-unis"),
    ("royaume-uni", "royaume-unis"),
    ("allemagne", "allemagne"),
    ("angleterre", "angleterre"),
    ("canada", "canada"),
    ("australie", "australie"),
    ("pays-bas", "pays-bas"),
    ("ghana", "ghana"),
    ("corée", "corée"),
    ("autriche", "autriche"),
    ("suède", "suède"),
    ("italie", "italie"),
    ("biélorussie", "biélorussie"),
    ("mauritanie", "mauritanie")
]

artist["Pays"] = cleanning(artist["Pays"], replace_words)
artist["Pays d'origine"] = cleanning(artist["Pays d'origine"], replace_words)

# Export

In [ ]:
for k1 in dfs_chart.keys():
    for k2 in dfs_chart[k1].keys():
        dfs_chart[k1][k2].to_csv(f"../chart/{k1}_{k2}.csv", encoding="utf-8-sig", index = False)

In [ ]:
artist.to_csv("../artist.csv", encoding="utf-8-sig")